# HAGrid, Provisioning and Deployment

In this lesson:

## Deployment Tooling

HAGrid can deploy to a large variety of targets and by leveraging other tools and formats this can be extended. The goal for `hagrid` is to be your friendly helper in navigating the world of deployment and provisioning the `syft` stack and to take a multitude of complex steps and turn them into a handful of uniform and consistent commands.

## HAGrid can be installed on

- macOS
- Linux
- Windows
- WSL2 on Windows

### Python

We chose to write `HAGrid` as a python cli tool to utilize the existing skills and tooling in our monorepo, however this brings its own problems to the setup process, namely we require:

- Python 3.7+
- Console Scripts in `PATH`

### Console Scripts

```python
# packages/hagrid/setup.py

setup(
    name="hagrid",
    description="Happy Automation for Grid",
    long_description="HAGrid is the swiss army knife of OpenMined's PySyft and PyGrid.",
    long_description_content_type="text/plain",
    version=__version__,
    author="Andrew Trask <andrew@openmined.org>",
    packages=find_packages(),
    package_data=DATA_FILES,
    install_requires=packages,
    include_package_data=True,
    entry_points={"console_scripts": ["hagrid = hagrid.cli:cli"]},
)
```

If you notice the part that says `entry_points` this is how you can call a python library as a cli tool. There's just one catch, you need these "console scripts" paths to be setup properly.

Remember in an earlier session when we discussed `PATH`?

This is something which does not always happen automatically with every python install. Currently it appears that installing `python3` with the Windows Store, does not create this entry to the users `PATH` meaning if they install a python library with a cli tool they can't use it without a bunch of manual steps to edit their `PATH`. 

Unfortunately, this means there are many more things that can go wrong with Windows and a higher likelihood that Windows users won't know how to easily solve them.

WSL2 is great, but in solving a problem it creates two new problems. Now users need to have a mental model of two different environments on their system which can have different versions of python installed and by extension different packages. We also can't rely on users having or being able to install `wsl2`, so in the end we need to support normal Windows AND `wsl2`.

## Additional Prerequisites

Currently `HAGrid` also requires:
- git
- ansible (for cloud vm deployments)

### Git

During development the fastest way to ship updated code to deployed nodes was simply to utilise the git repository. Of course the repository is large and this adds the requirement for the system to have `git` installed, which for example is not installed by default on `Windows`.

While we do now have `Continuous Deployment` builds of containers going to `dockerhub` there are several files still required to start the stack, namely the `docker-compose` files and several configuration files which can also require customization.

## Templating

We recently introduced a `manifest` format which will allow us to remove the need to have `git` installed while also providing a single group of minimal files required to start a node. This is something which platforms like `kubernetes` have had for a while in the form of `helm` charts, however no such single format exists for `docker compose` and we suspect that `Docker Desktop` will be a popular container platform for many of our users going forward for some time. So to solve this issue and provide the ability for `hagrid` to do variable substitution in configuration files we created our own `helm` chart like manifest and templating system.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://github.com/OpenMined/PySyft/blob/dev/packages/hagrid/hagrid/manifest_template.yml">manifest_template.yml</a></blockquote>

Once the templating system is finished being implemented and tested we can remove the need for `git` as a dependency on `hagrid`.

## Bootstrapping Problem

One of the hardest issues when providing software intended to run anywhere and everywhere, is normalizing the environment sufficiently that everything just works the majority of the time.

This is especially hard given both bugs in legacy versions of software that must be supported and in supporting the multitude of updates to all platforms that are generated on a nearly daily basis.

Due to all the moving pieces in setting up a `domain` or `network` server, we ended up building a set of tooling to first and foremost help ourselves during development and secondly smooth the setup process for our users who may or may not have the technical know how to configure and deploy container services on their desired target machine.

One of the biggest challenges here is to provide the smallest number of easy and reliable steps for a user to run which gets them to an automated setup process. Unfortunately because `hagrid` requires `python` and some `PATH` settings setup; and python itself can be a little finicky this results in a variety of strange bugs and issues that can arise during the bootstrapping process.

Once `hagrid` itself is installed and working we are generally in control and able to detect software and install things on behalf of the user as required.

### Alternatives

Since the majority of what `HAGrid` does, is execute system calls in the terminal and parse their output we could certainly switch it to something like `go` or `rust` which would enable small binaries without dependencies, at least on initial install, which could greatly ease the setup process.

Alternatively, if we can find a way to run some code first we can attempt to do some setup for users to ease this complication. We have experimented with installer scripts like this already.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://github.com/OpenMined/PySyft/blob/dev/packages/hagrid/hagrid/win_bootstrap.py">win_bootstrap.py</a></blockquote>

This Windows Bootstrapping python script detects various prerequisite packages including `choco` package manager, and optionally prompts the user to install them with `choco`, even working to manually escalate the user permission prompts as Windows lacks the `sudo` of `POSIX` systems.

Unfortunately this still requires python so for maximum compatibility it will likely need to be re-written in `powershell`, a task which no human should be reasonably subjected to.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://github.com/OpenMined/PySyft/blob/dev/packages/hagrid/scripts/install.sh">install.sh</a></blockquote>

Similarly, in the vein of many popular online installers some early experiments with a shell script that can be curled from the web was started. This should provide sufficient control for auto prompting install of some basic pre-requisites on common `*NIX` environments. There is a newer version in a pending PR so hopefully this will be improved shortly.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## HAGrid can launch on

```
docker
x.x.x.x (Any IP)
localhost
azure
gcp
aws (todo)
kubernetes (coming soon)
```

## Launch to Docker

So far we have mostly dealt with this target since it is the simplest. Remember, HAGrid takes a small number of options and inputs from the user and translates them into lots of terminal commands and network calls as needed to setup and provision a node.

In the case of the target `docker` this uses recent versions of `docker` and `docker compose` cli tools to do basically everything.

This has a few advantages because `docker` provides a consistent interface across many different oses and platforms and also allows us to mount source code directly into the containers for development.

For some commands we can pass `--cmd=true` and `hagrid` will print a dry run of what it was going to do. This is a handy way to understand what `hagrid` is really doing.

In [2]:
!hagrid launch test_domain domain to docker:8081 \
    --tag=latest --tail=false --dev --cmd=true


⠇ Checking for Docker Service     
✅ Docker service is running
✅ Git 2.37.3
✅ Docker 20.10.20
✅ Docker Compose 2.12.1


                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNmdmNNMMNNNy/osoo/-`
                    `-sNMMMMMMMMMMMMMMMMMMMMMMMMNNmmMMMMNh-....`
       

### Command Break Down

Let's break down what those commands above are really doing. The first part is a long list of environment variables which are being set before we run our first command.

```bash
RELEASE=development
COMPOSE_DOCKER_CLI_BUILD=1
DOCKER_BUILDKIT=1
HTTP_PORT=8081
HTTPS_PORT=444
TRAEFIK_TAG=51eb25e41d7a4b5351017dcb2feeafcfdde2a743d2407e5b7990b7407e2ea83a
DOMAIN_NAME=test_domain
NODE_TYPE=domain
TRAEFIK_PUBLIC_NETWORK_IS_EXTERNAL=False
VERSION=latest-dev
VERSION_HASH=495eb7dbe0e2ddc49e5818acab659206c7f74c25
USE_BLOB_STORAGE=True
STACK_API_KEY=Qn2lIlI0Q0B1aukBJchPRu4e2ykXsze7PbF6KL7IpOD54orh
```

Interesting, we can see that by choosing `8081` as our port that is being reflected in the `HTTP_PORT` environment variable. Additionally, the name of our `domain` is being set, the type of node is `domain` and we seem to have `latest-dev` as our version, due to the `--dev` flag.

If you aren't familiar with environment variables they are ways to expose `key-value` pairs to applications running on operating systems in a way which does not require the os or the application to implicitly define what they are during application compile time.

In `POSIX` operating systems like `linux` and `macOS`, these are usually written as `ALL_CAPS=value` by convention. On Windows there are two different syntaxes depending on if you are using `cmd.exe` or `powershell.exe`, for powershell it's similar to `POSIX` but with `$env:` prepended like `$env:ALL_CAPS="value"` with some other rules about `SPACING` and `"quotes"` around values.

```
docker compose
-p test_domain
--profile vpn
--profile blob-storage
--profile frontend
--env-file /Users/madhavajay/dev/PySyft/packages/grid/.envfile
--file docker-compose.yml
--file docker-compose.build.yml
--file docker-compose.dev.yml
up -d --build
```

There's quite a lot going on here so lets unpack it one flag at a time.

First, we are telling `docker compose` to use `-p` (project name) = `test_domain` which will prefix all the container names on the system. This is particularly useful for running more than one stack on the same machine for development.

The next three profiles are a way in `docker compose` to toggle containers on and off based on groupings. In this case we are saying this `domain` will have a `vpn`, `frontend` and `blob-storage`. You can see this by checking the compose files themselves.

```yaml
# packages/grid/docker-compose.yml

tailscale:
  profiles:
    - vpn
```

Removing this profile will prevent the deployment of the `tailscale` container for instances where a `firewall` busting `end-to-end` VPN is not desired.

The next line tells `compose` we want to use a custom environment variable file for this deployment where we can set some variables and secrets. This is a duplicate of the defaults in `packages/grid/.env` combined with the `ENV` variables that `hagrid` generates on your behalf and puts on the command line.

Unfortunately a version of `docker compose` broke command line `ENV` variables and so this workaround is required to maintain compatibility for now. If you check the `hagrid` source you will see how this file is generated on each run.

The next three `--file` directives allow for additional compose `YAML` files to be layered on top of the default `docker-compose.yml`. The way this works is each successive file overwrites all yaml keys that it defines. In this instance because we are using `dev` mode we need `docker-compose.build.yml` and `docker-compose.dev.yml` to help compose understand how to build the containers with our local code and how to mount the source code and configuration files for `hot-reloading` and open additional ports in `dev` mode.

### Hot Reloading

Let's see how hot reloading of the `backend` and `syft` is achieved and what some of the implications of it are.

If you look closely at the `backend.dockerfile` you will notice that we install `syft` in editable mode. This is so that modifications to the `/app/syft` path will result in the code which python uses for syft to change on every file save. Python is a dynamic language which means that code can be compiled at run time.

```dockerfile
# packages/grid/backend/backend.dockerfile

...

# install syft
RUN --mount=type=cache,target=/root/.cache \
  pip install --user -e /app/syft
```

Additionally within this special `docker-compose.dev.yml` file we can overwrite those internal paths in the container with volume mounts of our local `PySyft` code repo.

```yml
# packages/grid/docker-compose.dev.yml

backend:
    volumes:
      - ${RELATIVE_PATH}./backend/grid:/app/grid
      - ${RELATIVE_PATH}./backend/alembic:/app/alembic
      - ${RELATIVE_PATH}../syft:/app/syft
      - ${RELATIVE_PATH}./data/package-cache:/root/.cache
    command: /start-reload.sh
    environment:
      - PROFILE=true
      - JAEGER_HOST=docker-host
      - JAEGER_PORT=6831
```

Here you can see that `compose` will mount the path from your `PySyft` git repo checkout `/packages/grid/../syft` which is just `/packages/syft/` into the containers `/app/syft` directory where the internal python `site-packages` expects the `syft` code is installed with `-e` editable mode.

We then also replace the `command` with a version which starts our web server with `hot-reloading` using `exec uvicorn --reload`.

The final part of the `compose` command is `up -d --build`, where we ask compose to start our stack in `-d` detached mode and force a `--build` since in `dev` mode we need to assume any file including the actual `dockerfile` definitions themselves can change.

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Launch to x.x.x.x (Any IP)

Let's look at a different example to understand a bit more about what `hagrid` can do. What if we don't want to deploy to our `Desktop` machine but somewhere else?

Well if that `somewhere else` is an existing `linux` server somewhere running `ssh`, we can ask `hagrid` to connect to it over `ssh` and do all the setup for us.

The way this works is by leveraging a tool called `ansible` which allows us to define the state in which we want a system to be in, and have it run commands called a `playbook` against that machine until it gets into the desired state.

`Ansible` is only available on `linux` and `macOS` but it should be automatically installed with `hagrid` if you are on those operating systems. If you are on `Windows` you could try installing `hagrid` into `wsl2` which will support ansible.

Assuming we have the `IP`, and we have login credentials such as a `password` or `ssh-key` for a user on the machine with elevated priviledges, we can simply run:

In [5]:
!hagrid launch test_domain domain to 100.0.0.1 \
  --tag=latest --tail=false --username=ubuntu --auth_type=key \
  --key_path=/Users/madhavajay/.ssh/azureuser.pem --repo=OpenMined/PySyft --branch=dev --cmd=true



                                             `
                                         `.+yys/.`
                                       ``/NMMMNNs`
                                    `./shNMMMMMMNs``    `..`
                                  `-smNMMNNMMMMMMN/.``......`
                                `.yNMMMMNmmmmNNMMm/.`....`
                              `:sdNMMMMMMNNNNddddds-`.`` `--. `
                           `.+dNNNNMMMMMMMMMNNNNmddohmh//hddy/.```..`
                          `-hNMMMMMMMMMMMMNNdmNNMNNdNNd:sdyoo+/++:..`
                        ../mMMMMMMMMMMMMMMNNmmmmNMNmNNmdmd/hNNNd+:`
                        `:mMNNMMMMMMMMMMMMNMNNmmmNNNNNdNNd/NMMMMm::
                       `:mMNNNMMMMMMMMMMMMMMMNNNNdNMNNmmNd:smMMmh//
                     ``/mMMMMMMMMMMMMMMMMMMMMMMNmdmNNMMNNNy/osoo/-`
                    `-sNMMMMMMMMMMMMMMMMMMMMMMMMNNmmMMMMNh-....`
                   `/dNMMMMMMMMMMMMMMMMMMMMMMMMMMMNNMMMNy.`
                ``.omNNMMMMMMMMMMMMNMMMMMMMNmmmmNNMMMMN+`
        

### Command Break Down

**Note**: Everything after `hagrid launch test_domain domain to 100.0.0.1` is optional, and if not supplied, `hagrid` will prompt you to enter each answer on the command line. However, this won't work inside a jupyter notebook, so for this example I have manually supplied all required arguments.

It looks like all `hagrid` is doing here is changing directories and then running `ansible-playbook`.

Let's take a look at how the `ansible` command is built up.

```bash
cd /Users/madhavajay/dev/PySyft/packages/grid;
ANSIBLE_CONFIG=/Users/madhavajay/dev/PySyft/packages/grid/ansible.cfg
```

First we change working directories to the `grid` folder and then pass in an `ENV` variable which tells `ansible` to use a specific configuration file.

Next we run `ansible-playbook` against the `IP` we chose with: `ansible-playbook -i 100.0.0.1`. Ansible is designed to handle lots of machines at once from an inventory file but here we can tell it how to connect to just one machine.

The playbook path is passed in here `/Users/madhavajay/dev/PySyft/packages/grid/ansible/site.yml` which provides the instructions on what to do via a series of `roles` definied in `yaml`.

In the case of using an `ssh key` we pass in the path to it under `--private-key` and the user to log in with as `--user`.

If you do use a `password`, we detect that and use a special flag `-c paramiko` which uses a python ssh agent and supplies the password as an additional `ANSIBLE_ARGS`.

Finally, a series of `key-value` pairs are passed in with the `-e "key='value'"` flag allowing us to override defaults found in the `vars.yml` file at `packages/grid/ansible/group_vars/all/vars.yml`. This changes the behavior of the `ansible` playbook and those variables will be available to the remote machine while executing the playbook `tasks`.

Assuming the `credentials` match, ansible will log in and get to work.

### Ansible Roles

The `site.yml` file above lists a handful of roles which will be applied to the target machine.

```yml
# packages/grid/ansible/site.yml

---
- hosts: all
  gather_facts: False
  environment:
    LC_ALL: en_US.UTF-8
  become: yes
  roles:
    - node
    - jupyter
    - containers
    - update
```

Here we can see, `node`, `jupyter`, `containers` and `update`.

If we check the folder structure of `packages/ansible/roles` we should see:

```
├── containers
│   ├── handlers
│   │   └── main.yml
│   └── tasks
│       ├── containers.yml
│       ├── hagrid.yml
│       ├── main.yml
│       ├── src.yml
│       └── tls.yml
├── jupyter
│   └── tasks
│       └── main.yml
├── networkm
│   └── tasks
│       └── main.yml
├── node
│   ├── handlers
│   │   └── main.yml
│   └── tasks
│       ├── docker.yml
│       ├── main.yml
│       ├── security.yml
│       └── system.yml
└── update
    └── tasks
        └── main.yml
```

Ansible is about convention over configuration so each of those `roles` listed above will match a folder name and inside that folder will be a `tasks` folder with a `main.yml` which will get executed first.

```yml
# packages/grid/ansible/roles/node/tasks/main.yml
---
...

- name: Install docker
  import_tasks: docker.yml
  when: deploy_only is not defined and install == "true"
```

Here we can see that one of these tasks is to import and run another task for installing `docker`.

### Ansible Modules

One of the beautiful things about `ansible` is that while this has been mostly built and tested for `Ubuntu 20.04` it will be trivial to add support for most other `linux` operating systems as many of the ansible modules already support them automatically, by providing a uniform module system which substitutes package managers and service runners as required.

The easiest way to understand and work with `ansible` is simply search for `ansible module KEYWORD` and read the docs for any given module.

For example here is some code which uses the `package` module:

```yaml
- name: Install System Packages
  package:
    name: "{{ item }}"
    state: present
    autoclean: yes
    update_cache: yes
  loop:
    - acl
    - python3-pip
    - ntp
    - tmux
    - vim
    - ufw
    - git
    - python-is-python3
    - net-tools
    - ifupdown
    - python3-venv
    - cron
```

You can see the module name `package` and then the `name` of the package we want to install is a `jinja` template with the variable `item`.

This special variable comes from the below `loop` construct, which loops over all of the below inputs as a `yaml` array and executes the `package` module against them.

Searching for `ansible module package` will return this page:

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://docs.ansible.com/ansible/latest/collections/ansible/builtin/package_module.html">ansible.builtin.package module </a></blockquote>

Assuming the package names are the same between `distros` ansible will handle calling `apt`, `apk`, `yum` or `emerge` etc as required.

Check out the main `ansible` docs here:

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://docs.ansible.com/">Ansible</a></blockquote>

### Ansible Playbook

What does the `ansible` playbook do?

Other than installing packages and setting up `cron` jobs it basically just does two things:

```yaml
# packages/grid/ansible/roles/containers/tasks/hagrid.yml

- name: Install HAGrid
  shell: "runuser -l {{ om_user }} -c 'pip install -e {{ build_dir }}/packages/hagrid'"
  become: yes
```

```yaml
# packages/grid/ansible/roles/containers/tasks/containers.yml

- name: Start Docker Containers
  shell:
    cmd: runuser -l {{ om_user }} -c 'hagrid launch {{ node_name }} {{ node_type }} to docker:80 --release={{ release }} --tag={{ docker_tag }} --tail=false'
  become: yes
```

That's right! We're just installing `hagrid` and running `hagrid launch to docker`.

<img src="https://media.giphy.com/media/OK27wINdQS5YQ/giphy.gif" />

<br />
<table style="border:1px solid gray;" width="100%"><tr><td align="left" valign="top" width="50px;" style="padding-top:0; padding-left:15px;"><img style="margin-top:0px;" src="data:image/webp;base64, UklGRnwiAABXRUJQVlA4WAoAAAA0AAAAYwAAZQAASUNDUEgMAAAAAAxITGlubwIQ
AABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAA
AAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABFjcHJ0AAABUAAAADNkZXNjAAABhAAA
AGx3dHB0AAAB8AAAABRia3B0AAACBAAAABRyWFlaAAACGAAAABRnWFlaAAACLAAA
ABRiWFlaAAACQAAAABRkbW5kAAACVAAAAHBkbWRkAAACxAAAAIh2dWVkAAADTAAA
AIZ2aWV3AAAD1AAAACRsdW1pAAAD+AAAABRtZWFzAAAEDAAAACR0ZWNoAAAEMAAA
AAxyVFJDAAAEPAAACAxnVFJDAAAEPAAACAxiVFJDAAAEPAAACAx0ZXh0AAAAAENv
cHlyaWdodCAoYykgMTk5OCBIZXdsZXR0LVBhY2thcmQgQ29tcGFueQAAZGVzYwAA
AAAAAAASc1JHQiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAABJzUkdCIElFQzYxOTY2
LTIuMQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAWFlaIAAAAAAAAPNRAAEAAAABFsxYWVogAAAAAAAAAAAAAAAAAAAAAFhZ
WiAAAAAAAABvogAAOPUAAAOQWFlaIAAAAAAAAGKZAAC3hQAAGNpYWVogAAAAAAAA
JKAAAA+EAAC2z2Rlc2MAAAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAA
AAAAAAAAFklFQyBodHRwOi8vd3d3LmllYy5jaAAAAAAAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAABkZXNjAAAAAAAAAC5JRUMgNjE5NjYt
Mi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAAAAAAAAAAAC5J
RUMgNjE5NjYtMi4xIERlZmF1bHQgUkdCIGNvbG91ciBzcGFjZSAtIHNSR0IAAAAA
AAAAAAAAAAAAAAAAAAAAAAAAZGVzYwAAAAAAAAAsUmVmZXJlbmNlIFZpZXdpbmcg
Q29uZGl0aW9uIGluIElFQzYxOTY2LTIuMQAAAAAAAAAAAAAALFJlZmVyZW5jZSBW
aWV3aW5nIENvbmRpdGlvbiBpbiBJRUM2MTk2Ni0yLjEAAAAAAAAAAAAAAAAAAAAA
AAAAAAAAAAAAAHZpZXcAAAAAABOk/gAUXy4AEM8UAAPtzAAEEwsAA1yeAAAAAVhZ
WiAAAAAAAEwJVgBQAAAAVx/nbWVhcwAAAAAAAAABAAAAAAAAAAAAAAAAAAAAAAAA
Ao8AAAACc2lnIAAAAABDUlQgY3VydgAAAAAAAAQAAAAABQAKAA8AFAAZAB4AIwAo
AC0AMgA3ADsAQABFAEoATwBUAFkAXgBjAGgAbQByAHcAfACBAIYAiwCQAJUAmgCf
AKQAqQCuALIAtwC8AMEAxgDLANAA1QDbAOAA5QDrAPAA9gD7AQEBBwENARMBGQEf
ASUBKwEyATgBPgFFAUwBUgFZAWABZwFuAXUBfAGDAYsBkgGaAaEBqQGxAbkBwQHJ
AdEB2QHhAekB8gH6AgMCDAIUAh0CJgIvAjgCQQJLAlQCXQJnAnECegKEAo4CmAKi
AqwCtgLBAssC1QLgAusC9QMAAwsDFgMhAy0DOANDA08DWgNmA3IDfgOKA5YDogOu
A7oDxwPTA+AD7AP5BAYEEwQgBC0EOwRIBFUEYwRxBH4EjASaBKgEtgTEBNME4QTw
BP4FDQUcBSsFOgVJBVgFZwV3BYYFlgWmBbUFxQXVBeUF9gYGBhYGJwY3BkgGWQZq
BnsGjAadBq8GwAbRBuMG9QcHBxkHKwc9B08HYQd0B4YHmQesB78H0gflB/gICwgf
CDIIRghaCG4IggiWCKoIvgjSCOcI+wkQCSUJOglPCWQJeQmPCaQJugnPCeUJ+woR
CicKPQpUCmoKgQqYCq4KxQrcCvMLCwsiCzkLUQtpC4ALmAuwC8gL4Qv5DBIMKgxD
DFwMdQyODKcMwAzZDPMNDQ0mDUANWg10DY4NqQ3DDd4N+A4TDi4OSQ5kDn8Omw62
DtIO7g8JDyUPQQ9eD3oPlg+zD88P7BAJECYQQxBhEH4QmxC5ENcQ9RETETERTxFt
EYwRqhHJEegSBxImEkUSZBKEEqMSwxLjEwMTIxNDE2MTgxOkE8UT5RQGFCcUSRRq
FIsUrRTOFPAVEhU0FVYVeBWbFb0V4BYDFiYWSRZsFo8WshbWFvoXHRdBF2UXiReu
F9IX9xgbGEAYZRiKGK8Y1Rj6GSAZRRlrGZEZtxndGgQaKhpRGncanhrFGuwbFBs7
G2MbihuyG9ocAhwqHFIcexyjHMwc9R0eHUcdcB2ZHcMd7B4WHkAeah6UHr4e6R8T
Hz4faR+UH78f6iAVIEEgbCCYIMQg8CEcIUghdSGhIc4h+yInIlUigiKvIt0jCiM4
I2YjlCPCI/AkHyRNJHwkqyTaJQklOCVoJZclxyX3JicmVyaHJrcm6CcYJ0kneier
J9woDSg/KHEooijUKQYpOClrKZ0p0CoCKjUqaCqbKs8rAis2K2krnSvRLAUsOSxu
LKIs1y0MLUEtdi2rLeEuFi5MLoIuty7uLyQvWi+RL8cv/jA1MGwwpDDbMRIxSjGC
Mbox8jIqMmMymzLUMw0zRjN/M7gz8TQrNGU0njTYNRM1TTWHNcI1/TY3NnI2rjbp
NyQ3YDecN9c4FDhQOIw4yDkFOUI5fzm8Ofk6Njp0OrI67zstO2s7qjvoPCc8ZTyk
POM9Ij1hPaE94D4gPmA+oD7gPyE/YT+iP+JAI0BkQKZA50EpQWpBrEHuQjBCckK1
QvdDOkN9Q8BEA0RHRIpEzkUSRVVFmkXeRiJGZ0arRvBHNUd7R8BIBUhLSJFI10kd
SWNJqUnwSjdKfUrESwxLU0uaS+JMKkxyTLpNAk1KTZNN3E4lTm5Ot08AT0lPk0/d
UCdQcVC7UQZRUFGbUeZSMVJ8UsdTE1NfU6pT9lRCVI9U21UoVXVVwlYPVlxWqVb3
V0RXklfgWC9YfVjLWRpZaVm4WgdaVlqmWvVbRVuVW+VcNVyGXNZdJ114XcleGl5s
Xr1fD19hX7NgBWBXYKpg/GFPYaJh9WJJYpxi8GNDY5dj62RAZJRk6WU9ZZJl52Y9
ZpJm6Gc9Z5Nn6Wg/aJZo7GlDaZpp8WpIap9q92tPa6dr/2xXbK9tCG1gbbluEm5r
bsRvHm94b9FwK3CGcOBxOnGVcfByS3KmcwFzXXO4dBR0cHTMdSh1hXXhdj52m3b4
d1Z3s3gReG54zHkqeYl553pGeqV7BHtje8J8IXyBfOF9QX2hfgF+Yn7CfyN/hH/l
gEeAqIEKgWuBzYIwgpKC9INXg7qEHYSAhOOFR4Wrhg6GcobXhzuHn4gEiGmIzokz
iZmJ/opkisqLMIuWi/yMY4zKjTGNmI3/jmaOzo82j56QBpBukNaRP5GokhGSepLj
k02TtpQglIqU9JVflcmWNJaflwqXdZfgmEyYuJkkmZCZ/JpomtWbQpuvnByciZz3
nWSd0p5Anq6fHZ+Ln/qgaaDYoUehtqImopajBqN2o+akVqTHpTilqaYapoum/adu
p+CoUqjEqTepqaocqo+rAqt1q+msXKzQrUStuK4trqGvFq+LsACwdbDqsWCx1rJL
ssKzOLOutCW0nLUTtYq2AbZ5tvC3aLfguFm40blKucK6O7q1uy67p7whvJu9Fb2P
vgq+hL7/v3q/9cBwwOzBZ8Hjwl/C28NYw9TEUcTOxUvFyMZGxsPHQce/yD3IvMk6
ybnKOMq3yzbLtsw1zLXNNc21zjbOts83z7jQOdC60TzRvtI/0sHTRNPG1EnUy9VO
1dHWVdbY11zX4Nhk2OjZbNnx2nba+9uA3AXcit0Q3ZbeHN6i3ynfr+A24L3hROHM
4lPi2+Nj4+vkc+T85YTmDeaW5x/nqegy6LzpRunQ6lvq5etw6/vshu0R7ZzuKO60
70DvzPBY8OXxcvH/8ozzGfOn9DT0wvVQ9d72bfb794r4Gfio+Tj5x/pX+uf7d/wH
/Jj9Kf26/kv+3P9t//9BTFBI5wMAAAGgRtvWIUl6oWTbtm3btm3btm3btm3bto3M
qlTE6x5Exhffi/8RMQGgv4AD6I/6cYJCX2HE5iJ1wgREXzHqlIOI+D4bcY7riIgf
ctHmevgvfFmKNPd/4bfOMmWP/gsDq2OGJbvjp82aI3fuVLEdImeu+/+D2sM68v8o
TnvUYkO23/kY4fMH/L9fX1zaPqtNAZEb5cD/IWqr0ksAYG9w/c39FyqGGTo9q0nZ
RBInwlA9iJ7xiRXbdBUN9s3tmNIl8tFJH6L/+GwNjY/onjcrF1FfhIOoIUv1ZTwu
nDfDY/0kFQ/iVLPhm+wcQCbVbPggBQfSXtPh/qgmkySACqrp1L6CmaRqG7f1zLBJ
Mx3+TGOmEn5E1JDHqWaqoyKvTx0mivWYG3WCYB5Iv/Smxgf61leMo7ARdICUKYIT
xMCnO4dXjepdv2KeNMnjR4vitEnifwmSPUaWjnt76ADbXm7+Vw35f397/+TBtfMH
NiyYOmXKtAXr9lx64dE+ZddTO8ib4d5qoDPbF6Q52EvSEe00Ej1CBJ1jNJrUWRLo
zPobSQ71FkHvYiT5ay0B9Mb+QNLdXALonqQRFFyUFPTL95He1+0UCDP5L3L8u1ND
2O1VYoKXqkkQ/gEk1X+tqghG3qPEszSDDMa+4ke72KPThOV7T5w8d/7C5QPLB9fN
Gk0Ao9/xg/htmCzKsqIoNpssCsDyPU/oKQImfcxVZBmzbOTqtt0s5f0cBdvYHQ6X
O0q0WPFiMbJd0/gJvXz19sPnr99+eHw/m7CBGNv9oVAoGOJB/3cHG5BiJEyUoNg6
H2ehKoxAiNvuRqSGvF+TmcjlNr8JIYFqfhaVrwSQyJ0244SHSGZkWuNSBOjAcYJR
Ql+NEE8So2yPkNJWRsUOkrJdMKghkhqZ0xj7LVpwkWBI6iAxrxIZ0hCJVdsbIWyk
Bg8KBsT5Roz2ugYYWAtp1U7EBwPFJcRMd4KRMR8RUwUMzR+wggFoBRcswWcJazYf
OHP9uUZHRWMABFHO6CUPANIQUo5BSg8dZRik9dJRikHWSDpKMMjro6Mog6IBOoow
yOejoxCDvJZQ0G8FRSyheMAKSgWtoIwlVApZQW3VChpYQlNLaEVHKC+DjnT0kxh0
14gINJGAYX8ivlYSgOVwGt4UBbZjSbibFxhPRQIvpgDWC/jTDiQB5su4U7dHA/bL
edNmKUBeaIgCZlzGV2QHEUyZp+foEYOHjJywZPHS/UdOnD934fqDF59//PgdGVQ1
dj/qiGB6UZRkWVbszqixYsVPljZDjryFipWqUK1O09Ydu/XqN3jUpGkz5y5YumHj
ho2bt2zbvGHDkpJgQgBWUDggeg4AALA6AJ0BKmQAZgA+MRSIQqIhIRXrHvQgAwSy
AEJCK+91/JT8gPlFtv96/DvG/F97Ov4PrP/y3qv/vX7Y+4B+tfSi8wn68fsr7x/o
Z/tfqAf2X0rv+h7EPoLfth6bH7nfCR/b/+f+5ntO//f2ANc3/o3af/c/yQ/aT1j8
PPi7205E3Pfip+2H47+2/t367f77wx+D+oF6c/xu/H53/nvQC9p/oP+7+4j0NP8j
0M+uv+l9wD+U/07/Ofmr6wHg3+Pf6D3AP5X/Rv9T/a/24/0/x3f8P+N89f0J/2f8
j8BP8o/p3+0/uX+S/8X+g///1l+ub9nv//7oP60f/ge+6nLHlXthQI0YCULCJ67g
Zit4W3QneQ2KXjVNaPM0QH2FTGE1m/zFVScSOwbgo8koZky1bTPe7UbyFX4TFX7m
J9GIc2Toeknlk5GUrIh5RUvZAXIp/gWFhTi+j1hsJPtMeUFNSaJCXtbZCSP4TW6H
9waRceufIvLjbKbpZZjgfXX921byCVvvg2b38K1WXfjZvkeJkQJCvZL/dipnXoRU
7Hnbzw55UUC3Wm9qJDg5tGpqf8XdrnhVt6RK1Y1OXZ+9mjOwWW4LwxWTQVqwhj/J
2LfFWsfhpgORCBt35xYL9zxrdMgA/v+GKqZS1ziiCybfC3dpeCGH5xbdKaq9HHCW
dkRYZP6oshhZjbBttYFtLpxnfT9v7R5o38yJgJy3S69/whP7s78RueeY/vuhwh+f
hTZ7uWMzPkvRYmWf1cKEbYPgT2f9LpGbG/+Dtn3Nw4noBBxS4CEiqqBS/8UXvDsU
1n54rfaYcHWsteKu0u+cGJLPn8ukMia6iDXW0di2IuzSkhUh33N+u0fCAfRnakD9
qwi/bsXE1lpruxNL39iyDzP6dzVncfueK00CbCR1AknrvhL09G6faIVbQYXrj/9m
EwOyc9kNlFklC4rTmnz7dh3Ur9nxHXgHWn0dia59uw/ihIf1SqbmpItg1k/KKHhN
kQAvjCrTB8oRL9T/xvJv8XvWTm2GvNpVtYe6DtKL/LLYxS9UEI5DQb+qxWUzcDSq
ZTUmzhc3CrG4cujsTEWxxSJfTmMqBr3EsuygLIq3C6SizqRVMRXzD/3qC6DJMOda
wj1BP4Xg+O9MPDtNq9yGxGO11M3zvDTPzNgPc/to/msUkzH0Hu9Ee6B2ZqKhyLwb
93XLpuqnF9/aJiVegMXvq9u4GSZl6UQJ/ynbqe24uzsYprvPIbt9XUOvrZyOI2UY
XKEpOPp09JzCkcJ5xQjGY1cuRqZnluX8Bn3nBqumDB4efND8PrwWCi5SaUHC+zVW
+757EXSZNbEZecd36srMBJdmLTTB6MvEy5CaaXuGa5gnTXGgNCuoSk3AtTbElpC8
GF3oX05nB9Dw1bTCuBl/g84d66mye+NPwiB4cMd2PHfSxvfmd9pzV9oB7qbCSmMs
Ux5z/cMRdWS6nbj0E5XGwrOwE/breFHkNNK3WTFhKkZ/KGokpeIH+HfUnAp6gTVM
rLy2bf/+N/FeqZut1tIchmi6lYulh+gTwpGtnU75VsgSKa01vFsdsx6UnelXZGdt
S+NwxEMxWmy+Vk1qDHnbQ+jFZdWuMd4MiCEMbY/s83e7rkMQMadI7xfPW9xqZAF+
IRiO41AQU319Ck22XRrFohOHWMVdUoqHuB2+3PrpwwOCmmQsZ4i4tEKntQGC3SXu
RkAc/413+G8VRk+jD5YM+rwjvXKnMv3g9+e4pOwN9rDEOxKvL8XSSvd15+x9rG9j
HfuEppJMejLGoJ1U6HnoJ6cXTfi7jOTrgJlKeKKw1yBhelEvpPsYsjf74GfX/aQC
E55ObScjR5tk83M9+vFmgDSOhTSbuIxKiIaVerYP6v0XDxOD1wrL4H88zWCl6CKd
UGNpPHO0QESmmXqGkbKKDqIZdU82li2MFWjfo0uoJzKL5yHwrflaAf6bpW4rbbQt
Y1+PdUSMiSP5Hlyg8Cx+bZcwGu2Tp2DfhgOnO5GegvyM/4mufgexKiLf7EUp1w7K
bAOmGI+yR+urvMsjr6XNJT54RlPMm967Ab0jrb2VabVmLD/g+aEntati7UPpS/fE
BeBfX0JfFZ7f5qWmPgGRSQDpvuGbJ3ctQs4AwPIzuYQnd/ITsT3tma3f/+DrjyKp
fsgLn8ke7JJNOZFqhtfzGCeSs2tUhmQT5OAm4Vck1+6G2EHDSM/f5iT0vxAcplA0
7ULMjSxo2YWTM4pN/sw4/OmI3q178ussx9IlL75s4nQo0iHDA63LYjd93rtjZfwf
h2lWqgJeuewIiw+eLmnecQ5/3HbBGtqvdQo7SEUpyqWji1uujL0e3rrZCXGc9geh
kdGNW5INOyU0qa/E7PUouPPCWMCxdw85xsPvKLKxotSjwJrO7SVzvDilkALSg6PD
ZREg1tulqTGSV/M0qAAI+lBmHf14/icijyZKbsyMRM4kyonk9IC2SaaEV0LF6hjm
2A4wIV1RbEmy6htPOV36CX6G5oOqra5gV9vE5OOln8XWYBCSt90lc5XTqUb0PX5+
Q95MQfJDfm8vlikHED4+7E8ChwRsDWh8HWzt02mQjW3+ukWR5cdYuz1P0Tm0PqdQ
JYTwUyogv0m0IWzHGJtc3Zv2V6brAMhdYSFfP4rlDtc5IdU51irioyDuOQdF0mS1
g3Qxxtgk5r3Il/MPXi2Jzz5KpdryMgbO3gp17X8gSi/aLtCkRhykl9hmGg0N4vTr
R5ROLhBZSmp/nsc8PYDBeFs1+oB/7iKUl7oT6ZY9a4oSgSeoe8Wv5LOAHaFVj7Qd
fMOkoTCKejL/o3hJWJeE5ZgvHVqv+vhgOs+nvtTVflis9JBwwrBMLsm0sdfr6G+b
LAwMRtPD4LkG7v/lYG+oA8DoH7/ybPbKZgzX3p0Pyuf2r2y56TjXxR1eQZTp/uCd
ugTNzSXkK0U2H/4pa3n5t8UlsO7bOKQRpiN8S2nK1kKglJuN4jod1Au+Pp2A0w/4
IC/9AzVuLNDw9GSFF2kx2l4mbtrZjT0wManTw8FJbdc9QvpKghAuIKssj7uJUGUq
nDMfjguiIQla7ipCLKQArVK17iaxZH8lz+NhPu3rxWhn0z8bp3oaPQ0C/PD5/hSC
DynutQcMUEvWjU6J7Kmyy3a51ZAtJnnuNH+eGmZ4I/DDEpBKtlDwCYKWS+zem3T9
iTNgFM13PMCILdQ49P7a3s286aLAda/559sANwBSyJyhC9ugQ+vpfN1qgebemdiw
JqhL+GPi6VDuz+0owmjTbcDbAl+s/rDQ2wFdpYawzbEuFp8MTg2zkrBbaBHz0JBv
ue1ncslZH/lT9IsuuV6AHhNS/m0WrMG8+BJWQSas8SiG42N70dQP85Fq3jMebprd
XPgguMoUgh7xaJECjBmVqiMT8//Qgh+TU6S/m9Px33+jjAEBMrgFkQZryTbfsamj
w/zLRXHbHk4Iy3jSosAP/5/8GLx1xuqqzbsf/oMKwignPwSqvWTZu+L5orbP6wMj
nLy48mNhOczizQaNQJC9grENf44XKBViPlhkwHKQ7uf5iMFrw32+rHwPNPoqtapq
diCXBBWf/qTOaKI9P0U/nqcZZeG7em9aS2Qm6aCXPR8xWR4L+L4MgRTcQfOAl8hb
LgbEoU+cF5MN6gAwf+v+ezLYUfGCb3EhnPI0+yoNU34sKUYeR4o7dVYmNZ4RTGTU
lphMBKdQzn6/6r2+ELtIz3gt22MsbpmBohXcDwioh743kcSddYYDueetDTznaqmg
NsE8FC/C3UxlIHvurd18XwmlyUV6UXMS+02eOLJI7JKnwOBx4A6K5fWtwOukEigb
11EjgPyT7Bnww4n+OtEnugAWFWv7phQ3Il9DtxixptOp62JHKTH0878P/JM65KC9
Uj8OtupqLg0YuoTHSs0asAqrg7cEZBime44NFlT/0DY+uL2WVYYiDr8hrA7Qf7f0
oQyxmv77SrygbWQX4xksCXS8kFZMGiDffFbPNk7MiMx19v8zI1eJYT9zGbDLlGce
c0/qTle7WuKUwNOICsmwKxXIy22k9zJuZQSv3k/zYoeuhDyb5cc9vyh6qyPb9EBX
y3v/nEfuZdCqYzWvydYFinbJWIwf02/UGwCJczgkR7HbkFoV7ePzY09L1oIl6YrQ
mYusmif6kZRGoJfBjqBdp75U7cnN/kjARaFgHo/dnz7Humc7QpIadqX/nWgz/VLt
Zy+SN0B/+ET+qhSPD5TiuCz1WHKup/yX8P+FZpOe9nMMzMlprUKzmE9o8OmBI6fk
vAtbMMqnnPmWWJVP5Exq/nOCtf0nlWy/g5VSEYPmko8X34tq56AGL4RyapW0TgN+
iTxKoFDT8PN31qk5+LHC2cVDfLXz1h0+5E7uuHjUE6WsU4GBeDig9Su4K/7mcn8Y
aF1X3ce8wM3iTlf+Brqpn1kiF89T/YJhZE2bMjpeLoFD742NnvP2dNLhoHGhIova
uuwN72hVjg9WxX64HWjA9R5CnGXCrr+4tE22AXkypm+mGjCaRALDF9TH8ZqgzyHW
9S05vgfui6gbe4sAtXMS6+KGpNhC6rCxhjUWk164a2uv6KxQ3HS/7jhWppxFuB5P
AdhXOyYxPZ8GO5+UFBhfxOLaIpYPZTX0CfVDzJZXBfIB7zPZo2YNPkzTIrFRX8B7
+hTGvf29UQk8z1w7xwuzG66YUBc8e4u7yXbmimq6LeKiITz39GucMD68Fz49HNMX
kn3NkU2mpfwArLh2307tNXS1C++e3Rwv1ftV0DTYav6qhaeE6370/nTKiDlxqGvX
lxkITueTbmSFjfrdtWchhiuCnvMR5w7RpUH8Ej0Fg6vwh6rKUtQgj1WNpuNOjsjV
49uy/D++v4MDQwBQK0H4WQ+an1iaMbSZ0wWZnKJPVmaGLjmpeuxb/qaYOFhgELYx
YHnxajqWmAor8B7R9Ak3TRz447iEtNzL8WobSgAAAABYTVAgnAMAADx4OnhtcG1l
dGEgeG1sbnM6eD0iYWRvYmU6bnM6bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYu
MC4wIj4KICAgPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8x
OTk5LzAyLzIyLXJkZi1zeW50YXgtbnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlv
biByZGY6YWJvdXQ9IiIKICAgICAgICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25z
LmFkb2JlLmNvbS90aWZmLzEuMC8iCiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0
dHA6Ly9ucy5hZG9iZS5jb20vZXhpZi8xLjAvIgogICAgICAgICAgICB4bWxuczp4
bXA9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC8iPgogICAgICAgICA8dGlm
ZjpZUmVzb2x1dGlvbj43MjAwMDAvMTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAg
ICAgICAgIDx0aWZmOlJlc29sdXRpb25Vbml0PjI8L3RpZmY6UmVzb2x1dGlvblVu
aXQ+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0aW9uPjcyMDAwMC8xMDAwMDwvdGlm
ZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlm
ZjpPcmllbnRhdGlvbj4KICAgICAgICAgPGV4aWY6UGl4ZWxYRGltZW5zaW9uPjEw
MDwvZXhpZjpQaXhlbFhEaW1lbnNpb24+CiAgICAgICAgIDxleGlmOlBpeGVsWURp
bWVuc2lvbj4xMDI8L2V4aWY6UGl4ZWxZRGltZW5zaW9uPgogICAgICAgICA8eG1w
OkNyZWF0b3JUb29sPlBpeGVsbWF0b3IgUHJvIDIuNC43PC94bXA6Q3JlYXRvclRv
b2w+CiAgICAgICAgIDx4bXA6Q3JlYXRlRGF0ZT4yMDIyLTA5LTIyVDEwOjU4OjA1
KzEwOjAwPC94bXA6Q3JlYXRlRGF0ZT4KICAgICAgICAgPHhtcDpNZXRhZGF0YURh
dGU+MjAyMi0wOS0yMlQxMToxMDoyNisxMDowMDwveG1wOk1ldGFkYXRhRGF0ZT4K
ICAgICAgPC9yZGY6RGVzY3JpcHRpb24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1l
dGE+Cg==
" /></td>
<td valign="top" align="left" style="padding-top:0;"><h3 style="margin-top: 10px; margin-bottom:5px;">Discussion Time ⁉️</h3><p>If you have any questions, feel free to ask.<br />If we don't know the answers now, we will find them out after.</p>
</td>
</tr>
</table>
<br />

## Launch to localhost

There is a seperate target in `hagrid` called `localhost`. All this does is use the same code path as the above `launch to x.x.x.x` except it passes in `--connection=local` which has the effect of telling `ansible` you won't need a username, password or ssh-key, just run locally and use `sudo` as necessary.

In fact we use this inside an older auto update `cronjob` mechanism which was used to keep our `Ubuntu VM's` up to date with the latest syft code. This mechanism is a little error prone and sub optimal and we will likely be replacing this with something like rolling updates with `kubernetes` instead.

```bash
# packages/grid/scripts/redeploy.sh

...

HAGRID_CMD="hagrid launch ${7} ${6} to localhost --repo=${2} --branch=${3} --release=${RELEASE} --tag=${DOCKER_TAG}"
```

## Launch to Azure

Now that we know how `hagrid` can setup a linux machine for us, let's look at how we might use this to deploy a `cloud vm` onto `azure`.

Unfortunately, we can't just run `--cmd=true` here because the `azure` deployment runs multiple steps and we haven't added code to allow for a dry run (though this wouldn't be too difficult).

However, if you have an `azure` account setup and run the following command to see how it works.

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
hagrid launch test_domain domain to azure
</blockquote>

First, `hagrid` will check if the `azure-cli` tools are installed. The `azure-cli` tooling has many python packages and they are hundreds of megabytes so we don't install these by default.

### az cli tool

The `azure-cli` tools provide the command `az` which we can use to call from `hagrid`.

Once the `azure-cli` tools are installed, we run `az account show` to see if you are already logged in. If not, we prompt a web login flow allowing you to authorize your `azure-cli` install to access your `azure` account.

Once the `azure` tooling is ready we will prompt you for some additional questions relevant to `azure` vm deployment.

### Azure Questions

HAGrid will ask you additional questions to gather the additional information it requires to complete the deployment to `azure`.

You may notice that questions sometimes come with a default value, and that once you input a value that becomes the new `default`. This is handled through a `json` file which we write into your `HOME` directory.

```json
// ~/.hagrid/cache.json
{
  "azure_resource_group": "mygroupname,
  "azure_location": "westus",
  "azure_size": "Standard_D8s_v3",
  "azure_username": "azureuser",
  "azure_key_path": "/Users/madhavajay/.ssh/azureuser.pem",
  "azure_repo": "OpenMined/PySyft",
  "azure_branch": "dev",
  "username": "ubuntu",
  "auth_type": "key",
  "key_path": "/Users/madhavajay/.ssh/azureuser.pem",
  "repo": "OpenMined/PySyft",
  "branch": "dev",
  "install_wizard_complete": true
}
```

`What resource group name do you want to use (or create)?`

Azure has the concept of a `resource group` which contains multiple cloud resources and is tied to a specific `region`. It is a good idea to deploy to something unique so you can easily manage and delete the entire `resource group` as needed to be sure everything is cleaned up.

`If this is a new resource group what location? [westus]:`

The next step is to select the `region` which is required if we need to create the resource group on your behalf.

We can see what our options are by running:

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
az account list-locations -o table
</blockquote>

```
DisplayName               Name                 RegionalDisplayName
------------------------  -------------------  -------------------------------------
East US                   eastus               (US) East US
East US 2                 eastus2              (US) East US 2
South Central US          southcentralus       (US) South Central US
West US 2                 westus2              (US) West US 2
West US 3                 westus3              (US) West US 3
Australia East            australiaeast        (Asia Pacific) Australia East
Southeast Asia            southeastasia        (Asia Pacific) Southeast Asia
North Europe              northeurope          (Europe) North Europe
Sweden Central            swedencentral        (Europe) Sweden Central
UK South                  uksouth              (Europe) UK South
West Europe               westeurope           (Europe) West Europe
Central US                centralus            (US) Central US
South Africa North        southafricanorth     (Africa) South Africa North
Central India             centralindia         (Asia Pacific) Central India
East Asia                 eastasia             (Asia Pacific) East Asia
Japan East                japaneast            (Asia Pacific) Japan East
Korea Central             koreacentral         (Asia Pacific) Korea Central
Canada Central            canadacentral        (Canada) Canada Central
France Central            francecentral        (Europe) France Central
Germany West Central      germanywestcentral   (Europe) Germany West Central
Norway East               norwayeast           (Europe) Norway East
Switzerland North         switzerlandnorth     (Europe) Switzerland North
UAE North                 uaenorth             (Middle East) UAE North
Brazil South              brazilsouth          (South America) Brazil South
East US 2 EUAP            eastus2euap          (US) East US 2 EUAP
Qatar Central             qatarcentral         (Middle East) Qatar Central
Central US (Stage)        centralusstage       (US) Central US (Stage)
East US (Stage)           eastusstage          (US) East US (Stage)
East US 2 (Stage)         eastus2stage         (US) East US 2 (Stage)
North Central US (Stage)  northcentralusstage  (US) North Central US (Stage)
South Central US (Stage)  southcentralusstage  (US) South Central US (Stage)
West US (Stage)           westusstage          (US) West US (Stage)
West US 2 (Stage)         westus2stage         (US) West US 2 (Stage)
Asia                      asia                 Asia
Asia Pacific              asiapacific          Asia Pacific
Australia                 australia            Australia
Brazil                    brazil               Brazil
Canada                    canada               Canada
Europe                    europe               Europe
France                    france               France
Germany                   germany              Germany
Global                    global               Global
India                     india                India
Japan                     japan                Japan
Korea                     korea                Korea
Norway                    norway               Norway
Singapore                 singapore            Singapore
South Africa              southafrica          South Africa
Switzerland               switzerland          Switzerland
United Arab Emirates      uae                  United Arab Emirates
United Kingdom            uk                   United Kingdom
United States             unitedstates         United States
United States EUAP        unitedstateseuap     United States EUAP
East Asia (Stage)         eastasiastage        (Asia Pacific) East Asia (Stage)
Southeast Asia (Stage)    southeastasiastage   (Asia Pacific) Southeast Asia (Stage)
East US STG               eastusstg            (US) East US STG
South Central US STG      southcentralusstg    (US) South Central US STG
North Central US          northcentralus       (US) North Central US
West US                   westus               (US) West US
Jio India West            jioindiawest         (Asia Pacific) Jio India West
Central US EUAP           centraluseuap        (US) Central US EUAP
West Central US           westcentralus        (US) West Central US
South Africa West         southafricawest      (Africa) South Africa West
Australia Central         australiacentral     (Asia Pacific) Australia Central
Australia Central 2       australiacentral2    (Asia Pacific) Australia Central 2
Australia Southeast       australiasoutheast   (Asia Pacific) Australia Southeast
Japan West                japanwest            (Asia Pacific) Japan West
Jio India Central         jioindiacentral      (Asia Pacific) Jio India Central
Korea South               koreasouth           (Asia Pacific) Korea South
South India               southindia           (Asia Pacific) South India
West India                westindia            (Asia Pacific) West India
Canada East               canadaeast           (Canada) Canada East
France South              francesouth          (Europe) France South
Germany North             germanynorth         (Europe) Germany North
Norway West               norwaywest           (Europe) Norway West
Switzerland West          switzerlandwest      (Europe) Switzerland West
UK West                   ukwest               (Europe) UK West
UAE Central               uaecentral           (Middle East) UAE Central
Brazil Southeast          brazilsoutheast      (South America) Brazil Southeast
```

`What size machine? [Standard_D8s_v3]:`

After that its important to choose a machine size, if you are doing container building such as dev mode you probably want 8 cores however if you are just deploying the pre-built containers you can probably manage on 4.

Also note, we currently do not validate the `machine size` with what is available in the region of your `resource group` and azure has many different machine size names depending on what region you are in.

### Creating the VM

Once `hagrid` has all the information it needs to create the `vm` it will run the following commands on your behalf.

`az group create -l westus -n mygroupname`

This will create the resource group as per your specification if it does not exist.

The next step is to create a VM with a `ubuntu 20.04` image and the machine size you specified.

`az vm create -n test_domain -g 2022.11.01_test_group --size Standard_D8s_v3 --image Canonical:0001-com-ubuntu-server-focal:20_04-lts:latest --os-disk-size-gb 200 --public-ip-sku Standard --authentication-type ssh --admin-username azureuser --ssh-key-values /var/folders/4h/js8n7tdd58jbkn34_1nhlc300000gn/T/tmp0ht4vnwk/hagrid_azureuser_key.pub`

To ensure that your `ssh-key` is in the right format, we convert it if necessary and save the public key to a temporary path.

This step will send back information about the `vm` we have created including its `IP` which we parse for later use.

### Network Security Group

To allow the machine to be connected from the outside world we create some firewall rules.

`az network nsg rule create --resource-group myresourcegroup --nsg-name test_domainNSG --name HTTP --destination-port-ranges 80 --priority 500`

`az network nsg rule create --resource-group myresourcegroup --nsg-name test_domainNSG --name HTTPS --destination-port-ranges 443 --priority 501`

At that point we wait until the IP that `azure` returned is responding on the `ssh` port 22.

### Ansible Playbook

Once we have an `IP` and `ssh` credentials we step back to the previous `ansible` deployment system and resume from there.

`cd /Users/madhavajay/dev/PySyft/packages/grid;ANSIBLE_CONFIG=/Users/madhavajay/dev/PySyft/packages/grid/ansible.cfg ansible-playbook -i x.x.x.x, /Users/madhavajay/dev/PySyft/packages/grid/ansible/site.yml --private-key /Users/madhavajay/.ssh/azureuser.pem --user azureuser -e "node_type='domain'" -e "node_name='test_domain'" -e "github_repo='OpenMined/PySyft'" -e "repo_branch='dev'" -e "docker_tag='latest'" -e "release='production'"`

This will continue and work indentically to the above step `Launch to x.x.x.x (Any IP)` as though we had manually created this Ubuntu 20.04 VM and added the firewall rules ourselves by hand.

<img src="https://media.giphy.com/media/0NwSQpGY6ipgOSt8LL/giphy.gif" />

### Azure Images

We also experimented with creating a single click Azure Marketplace VM image.

<img src="data:image/webp;base64,
...
" />

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://azuremarketplace.microsoft.com/en-us/marketplace/apps/madhavajay1632269232059.openmined_mj_grid_domain_ubuntu_1?tab=Overview">Azure Marketplace OpenMined Grid Domain
</a></blockquote>

It is not recommended you use this currently as the image is quite out of date, however the code and processes we created to generate this image are documented and its worth looking at how this works as prebuilt images are a common way to deploy to a variety of different `cloud` and `on-prem` Virtualization platforms.

## Packer

Hashicorp who are well known for a lot of great `virtualization` and cloud `orchestration` tools also make a great little tool called `packer` designed to build `VM` images.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://www.packer.io/">Packer
</a></blockquote>

The way it works is you define a pipeline using their `hcl` domain specific language and which takes a `source` runs some `build` steps on it and then produces an output.

### Install Packer

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🍏 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
brew install packer
</blockquote>

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
choco install packer
</blockquote>

### Manual Install

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://developer.hashicorp.com/packer/downloads?host=www.packer.io">Packer Downloads
</a></blockquote>

### Setup Azure

First, create an `app` to use within the packer file, this will also be the name for where your image gets stored.

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
az ad sp create-for-rbac --name openmined-images > azure_vars.json
</blockquote>

This will create a file called azure_vars.json which will look something like this:

```json    
{
  "appId": "21b92977-8ad0-467c-ae3a-47c864418126",
  "displayName": "openmined-images",
  "password": "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
  "tenant": "e3f9defa-1378-49b3-aed7-3dcacb468c41"
}
```

You need to know your `subscription_id`, which on `macOS` and `linux` is easy:

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
SUBSCRIPTION_ID=$(az account show --query id | tr -d '"')
</blockquote>

### Build Script

We have a build script to make the steps easier.

```bash
# packages/grid/packer/build_azure.sh

packer init azure.pkr.hcl
PACKER_LOG=1 PACKER_LOG_PATH=./packer.log packer build -var-file=azure_vars.json -on-error=ask -var "subscription_id=${1}" azure.pkr.hcl
```

You can run it like so:

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
./build_azure.sh ${SUBSCRIPTION_ID}
</blockquote>

This will ask `packer` to run the `azure.pkr.hcl` file, passing in the `azure_vars.json` and our `subscription_id`.

We are using a special builder provided by azure called `azure-arm` (nothing to do with ARM CPUs).

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://developer.hashicorp.com/packer/plugins/builders/azure/arm">Packer azure-arm
</a></blockquote>

Let's take a look at the `source` first. 

```json
// packages/grid/packer/azure.pkr.hcl

source "azure-arm" "domain" {
  azure_tags = {
    node_type = "domain"
    os_version = "ubuntu2004"
  }
  client_id                         = "${var.appId}"
  client_secret                     = "${var.password}"
  image_publisher                   = "canonical"
  image_offer                       = "0001-com-ubuntu-server-focal"
  image_version                     = "latest"
  image_sku                         = "20_04-lts"
  location                          = "West US"
  os_type                           = "Linux"
  subscription_id                   = "${var.subscription_id}"
  tenant_id                         = "${var.tenant}"
  vm_size                           = "Standard_D4s_v3"
  os_disk_size_gb                   = 128
  # vhd
  # capture_container_name            = "openmined-domain-ubuntu2004" # vhd
  # capture_name_prefix               = "openmined-domain" # vhd
  # resource_group_name               = "openmined-images" # vhd
  # storage_account                   = "openminedimgs" # vhd
  # managed image
  managed_image_name                = "openmined-domain-ubuntu2004-5" # managed image
  managed_image_resource_group_name = "openmined-images" # managed image
}
```

Most of that is just `azure-arm` stuff to say we want to use that specific `ubuntu` image on their servers and that `machine size` and `disk size` and that the name of the image will be `openmined-domain-ubuntu2004-5`.

Next we can see the `build` steps:

```json
// packages/grid/packer/azure.pkr.hcl

build {
  name = "openmined.node.domain"
  sources = ["source.azure-arm.domain"]

  provisioner "ansible" {
    playbook_file = "../ansible/site.yml"
    extra_arguments = [ "-v", "-e", "packer=true", "-e", "repo_branch=0.7.0" ]
  }

  provisioner "shell" {
    expect_disconnect = true
    scripts           = ["${path.root}/scripts/update.sh", "${path.root}/scripts/motd.sh", "${path.root}/scripts/hyperv.sh", "${path.root}/scripts/cleanup.sh"]
  }
}
```

Here we can see that we reference the above `source` and then go on to choose a `provisioner`. Packer provides support for a variety of provisioners including our beloved `ansible`.

You can see we are referencing the same `site.yml` playbook as before and passing in some extra vars such as `packer=true` which we can use to trigger different steps inside the ansible `role` files.

Finally, there are a bunch of `shell` scripts I found on the internet which do a few things such as install `hyperv`, update the kernel and add a `motd` (message of the day).

After we run this, `packer` will launch a `vm` running `ubuntu 20.04` on azure for us and point `ansible` at it and run the same playbook as the above examples. When it finishes it will snapshot that image and prepare it to be used by any VM on `azure`.

Yup, the same `ansible` code which can be flexibly changed to build and support other linux distros, which installs `hagrid` which knows how to configure and deploy a node to a `container` host of choice.

<img src="https://media.giphy.com/media/SDogLD4FOZMM8/giphy.gif" />

### HAGrid Custom Image

*But wait theres more.*

Azure can be told to use these images and we can ask `hagrid` to pass that information on. Currently this is a little hidden but we have a flag called `--image_name` which can be run with `domain_0.7.0` and the following image will be used: `madhavajay1632269232059:openmined_mj_grid_domain_ubuntu_1:domain_070:latest`.

Unfortunately this is a little convoluted, if you build your own image you can use it with your own `azure` account but if you want to share it with others publicly it must be submitted to the `Azure Marketplace` which can take several days and then the `sku` / `offer` name changes to something public like the one above.

The benefit of this is to have `hagrid` be able to deploy prebuilt images to `azure` fast. Other cloud providers have similar features we can easily support in the future as desired.

### Multi-Node Deployment

*Call in the next 5 minutes and receive not just 1 domain, but `n` domains for the same low price.*

HAGrid also supports the ability to launch multiple nodes on `azure` at once using the `--node_count` arg.

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
hagrid launch to azure --image_name=domain_0.7.0 --jupyter --node_count 10
</blockquote>

This is something we developed to rapidly deploy large numbers of Domain `vm`'s to Azure for some internal research projects involving over `100 nodes!` 😬

<img src="https://media.giphy.com/media/UvWllmm27ZaNbRpv2n/giphy.gif" />

## Launch to GCP

You probably guessed it, the way `hagrid` handles `GCP` deployment is to use the `gcloud` cli tool to create a VM and then to run the `ansible` playbook against that VM.

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
hagrid launch test_domain domain to gcp
</blockquote>

Let's look a little closer at each step.

Once the `gcloud` cli tool is available on your path, `hagrid` will run `gcloud auth login` to ensure you are authenticated.

At that stage it will ask a series of similar questions to the azure process.

Unlike `azure` which has `resource groups`, `Google Cloud Platform` has a similar concept called a Project or `PROJECT_ID`.

We will prompt the user to enter their desired project and then run `gcloud config set project PROJECT_ID`.

### Creating the VM

The next step is to determine what `zone` (similar to azure region) we want to run the `vm` in and what sized machine to use.

We also need to know what ssh user account to use and which key which by default is the `~/.ssh/google_compute_engine` key which the `gcloud` tool creates automatically.

To ensure the project exists we call `gcloud projects create myproject --set-as-default`

The VM is then created with the command: `gcloud compute instances create test_domain --project=myproject --zone=us-central1-c --machine-type=e2-standard-4 --create-disk=auto-delete=yes,boot=yes,device-name=test_domain,image=projects/ubuntu-os-cloud/global/images/ubuntu-2004-focal-v20220308,mode=rw,size=200,type=pd-ssd --network-interface=network=default,network-tier=PREMIUM --maintenance-policy=MIGRATE --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any`

Our current support for `GCP` has many rough edges and we will likely expand on it in the coming months, but ultimately if we can create a VM and provision it with `ansible` we can install any dependencies required to run a `container` host of our choice and then have `hagrid` do the rest for us.

Much less mind blowing at this point.

<img src="https://media.giphy.com/media/SbtWGvMSmJIaV8faS8/giphy-downsized.gif" />

## Launch to AWS

As you can imagine adding `aws` support in a similar fashion would be pretty simple and would make a great project for someone in the community to contribute.

## Launch to Kubernetes

We have started on providing the same facility to launch to `k3d` which allows for a small `k3s` kubernetes cluster running on Docker. Our `kubernetes` integration tests already do this albeit a little manually, the next step here is to get `hagrid` to do generate all the correct commands automatically.

Naturally we will be extending this to target native `kubernetes` clusters using `kubectl` and `devspace` but this will be a discussion for a later session.

## Desktop vs Server

The important distinctions when considering if a deployment is desktop or not is really related to who is deploying a node, which machine they are running `hagrid` on and what the `os` and `container` host will be.

For Desktop this might include:

- MacOS  
  Docker Desktop

- Windows  
  Docker Desktop

- Windows WSL2  
  Docker Engine

- Linux  
  Docker Engine

Of course other combinations are possible such as running the linux version of `Docker Desktop` on `linux` or possibly even `wsl2` but the principal remains the same.

For a Server install this is more likely to be:

- Linux VM  
  Docker Engine

- Kubernetes Cluster  

Other combinations are also possible for servers, there is nothing stopping someone running a `Windows` server or other container hosts such as `lima`, `minikube`, `podman`, `Rancher Desktop`, etc and we would love to get these supported and into our `CI` testing.

## Layer Cake

## Apple Silicon / Arm64 Support

Over the last few years nearly all `destkop` and `server` CPUs have converged on a CPU architecture know as `x86_64`. When code is compiled into a binary representation it generally must target both an Operating System which will know how to load and execute the binary code and a CPU architecture which will know how to run the operations that the code contains.

The `syft` stack makes use of a lot of compiled code both in the `container` layers and in the `python` libraries which contain highly optimized `math` and `tensor` code.

Unfortunately, this means there are some nuances that arise when supporting non `x86_64` architectures. This section will briefly explain what those issues are and how they are currently solved.

Since the release of Apple's new `Apple Silicon` arm64 CPUs the adoption of `arm64` has exploded and is no longer primarily a smart phone target, but a desktop target too.

### Docker arm64

Docker has gone to great lengths to make support to allow users to run `containers` built for different architectures with relative ease. This is done by providing upstream `base` images which have been compiled for different `CPU` architectures and transparently loading the correct one depending on what system the user is running.

```dockerfile
# packages/grid/backend/backend.dockerfile

FROM python:3.10.7-slim as build
```

You will notice in our `dockerfile`'s that there is no mention of a CPU architecture in the image name. Instead the parent images such as `FROM alpine` will have specific builds for each architecture and they will be downloaded and used where appropriate.

So what happens if you want to run `arm64` mode on your Intel CPU? Well you can do that but it will have to emulate the `CPU` which will slow down performance a lot.

If you are on `x86` you can run the following to test it out:

<table width="100%">
    <tr>
        <td align="left" valign="middle" width="20px"><img width="25px" src="data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAGQAAABkCAYAAABw4pVUAAAAAXNSR0IArs4c6QAA
AMZlWElmTU0AKgAAAAgABgESAAMAAAABAAEAAAEaAAUAAAABAAAAVgEbAAUAAAAB
AAAAXgEoAAMAAAABAAIAAAExAAIAAAAVAAAAZodpAAQAAAABAAAAfAAAAAAAAABI
AAAAAQAAAEgAAAABUGl4ZWxtYXRvciBQcm8gMi40LjcAAAAEkAQAAgAAABQAAACy
oAEAAwAAAAEAAQAAoAIABAAAAAEAAABkoAMABAAAAAEAAABkAAAAADIwMjI6MDk6
MjEgMTM6MDc6NDUA+UqCTgAAAAlwSFlzAAALEwAACxMBAJqcGAAAA7JpVFh0WE1M
OmNvbS5hZG9iZS54bXAAAAAAADx4OnhtcG1ldGEgeG1sbnM6eD0iYWRvYmU6bnM6
bWV0YS8iIHg6eG1wdGs9IlhNUCBDb3JlIDYuMC4wIj4KICAgPHJkZjpSREYgeG1s
bnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIyLXJkZi1zeW50YXgt
bnMjIj4KICAgICAgPHJkZjpEZXNjcmlwdGlvbiByZGY6YWJvdXQ9IiIKICAgICAg
ICAgICAgeG1sbnM6dGlmZj0iaHR0cDovL25zLmFkb2JlLmNvbS90aWZmLzEuMC8i
CiAgICAgICAgICAgIHhtbG5zOmV4aWY9Imh0dHA6Ly9ucy5hZG9iZS5jb20vZXhp
Zi8xLjAvIgogICAgICAgICAgICB4bWxuczp4bXA9Imh0dHA6Ly9ucy5hZG9iZS5j
b20veGFwLzEuMC8iPgogICAgICAgICA8dGlmZjpZUmVzb2x1dGlvbj43MjAwMDAv
MTAwMDA8L3RpZmY6WVJlc29sdXRpb24+CiAgICAgICAgIDx0aWZmOlhSZXNvbHV0
aW9uPjcyMDAwMC8xMDAwMDwvdGlmZjpYUmVzb2x1dGlvbj4KICAgICAgICAgPHRp
ZmY6UmVzb2x1dGlvblVuaXQ+MjwvdGlmZjpSZXNvbHV0aW9uVW5pdD4KICAgICAg
ICAgPHRpZmY6T3JpZW50YXRpb24+MTwvdGlmZjpPcmllbnRhdGlvbj4KICAgICAg
ICAgPGV4aWY6UGl4ZWxZRGltZW5zaW9uPjEwMDwvZXhpZjpQaXhlbFlEaW1lbnNp
b24+CiAgICAgICAgIDxleGlmOlBpeGVsWERpbWVuc2lvbj4xMDA8L2V4aWY6UGl4
ZWxYRGltZW5zaW9uPgogICAgICAgICA8eG1wOk1ldGFkYXRhRGF0ZT4yMDIyLTA5
LTIxVDEzOjA4OjIwKzEwOjAwPC94bXA6TWV0YWRhdGFEYXRlPgogICAgICAgICA8
eG1wOkNyZWF0ZURhdGU+MjAyMi0wOS0yMVQxMzowNzo0NSsxMDowMDwveG1wOkNy
ZWF0ZURhdGU+CiAgICAgICAgIDx4bXA6Q3JlYXRvclRvb2w+UGl4ZWxtYXRvciBQ
cm8gMi40Ljc8L3htcDpDcmVhdG9yVG9vbD4KICAgICAgPC9yZGY6RGVzY3JpcHRp
b24+CiAgIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+CqwsYWAAABMwSURBVHgB7V0L
dBzVef539i1pJUuy5AfYWI4hlrEJBbkYOOEQbGwcIClgJ7hpQ4EkdSiEpgmc07S0
Jj2UnKSE5gChOcUuTTEPIxsITuJQEk6BxKY2BGyrtkF+YFuWbGxL+5C0z5n+3525
szOr3dVKljdYO/dodu7c93zf/f//3jujuS4aO+cyilL4DL84f+6223yXX351KDSp
KRD0un1et1tJK4pMO3a1n4aSPKqqpTIZdTCVSUaPfhTf/puNsbVr1ya4Ko0P1XJG
7Qg7ZXeqwAB0z3/+8vXZk2acfZ3P77/EpShTFZer0eWiOhe5asjl8llbaq0Q4bi2
xlvvKDdOXueerXngRzycLF/6RaDxI8uwhsEv88Iv81vb59K0JF/HNE0Lq5p6Qs2o
3alkckvXhwd+fvs1V+3ibGk+QNaonLX+UgtAHvdPXt50waw55z/s8Xjm8W3UM/BZ
cDXjVjglEmvGrel+GWZUh7Sc13S5eY14mddMJzxGPTLQUpZIn5vXEi+z4CzTurgd
KFG4fHmRUv8z04m8fIW71zS1lwVqx4EP9tx725Kr3uZyMnyYReoFF/9FeSNxntW/
fHX+J2bP2cggNuiNsWfPF2ZPoV+Vkq5QmkLhsp588TJMnmXa3HO++HxhuflwLdOB
V9Z2PYc+6Fzx5cVX/JajIDUlOZRRilOuvPLK2vtWP/WeoijTkUFWbs3MYsz9geWB
D1XN0IljxyjS20vxwQFKp9Mi3Jpe9B1rQbmtkQIgz9bMMgxnuGJ5c+NlXoRb/biG
k2FG2S42eW6Ph4LBKqqtr6eGiU3iWmgFaAYcRjaR3/KjptN7n1rz+KX/8eCDJzh4
WFWWexuWokyv59nfbl3WNHnKWq6YbYbRSo5Op9LUfehDWv/kanplfbsA3sxVQR62
nXTNsi/SF7+6khgn8vh8lrvX2T20b9/1tyz69CbAZokc4i1KyKylS/0P/fO//FNN
qPYea85kIkH//eJ6euT++4gNmjWq4v0gZ+V3/oEW33ATBauqbXhEw30P/MnF8+7n
wJQtwnJRkJBZs2b5f9j+0g+qqkN3WdLTW6/9mlbduZKJwOjPcYUQCNXV0T3fe4gu
W3g1q9MszIP90X+7/sLz7+Z8eXtyNqW9ZE/7lne+XdfQ8KAM1lSNHvrOvfSrDc/L
IOdcAgLX3PQFuvu7D5DHCzXG6ov/In3hVTdd8qkHOGCI+spHiPLwU89den7bH78p
68tkMnTPl1fQzre3yiDnPAIE5rXNp+8/+bQxEOCMmpbu2t/5+VuXLoZNsRl6TOxs
rqmpqar1jy7ahJGSfqj091+71SHDhtLILnZs20rf+vObGU+Vx8PAlTxTpresWbJk
yQQuySYUtguO9Dy/edvjtXUTvoIqMT5Y++NH6L8e/REuR+zEcNDtZhXKvOfWNOLS
/sAZoG0AKGsLMbwfRXOW3/41+sq37jXHqbFw+NFll178N1yUaeStMMFft6njg15m
UOB3cF8n/eXnP1tyA/yhWgrU1olxudWQjaLtH/8shgaJR8KUiEZKai866Ia3fi9G
X+js7NRX2tvPfXjV3+5nvwiyqiz3+s1vPwGRYrkSveFHq+4riQyQMGHaORScwCso
CqTByrOoePz9YDLI94p7xr0Dg+EcJOvrN14vsAXG7FEuuerKb3I+j8xrJaTKH6z6
rBBLznig8wPqeGebTJf3jAbVTp5KgTqowsp2wKB26tm6ei4CRc/hQzyZPig6OrBm
87By8Q031MsskhDlX59+7momPSiN+bonfiLT5D1D/EJMhuL15o2vxECF7WVoylRd
ZRcB4LEH7jcIEcuunqU33nwZJxdcSFHxTG85905eERPFYPb92s9fLlIkUTWv56AB
hVzi+iipM9Pk+1k1ufdblxLsOTRFo/h5ffbAAle+bi4rXLisAtnKGgxMgE3so2MF
631/5w6sPprxk6dN+wJf/IKPpCTEp3iU8+ToIdLXaybO53HzJMcTCOaLMsNSC/sp
fUWCPO/4ihJCTEiipTSj6I56P/aEAABg4+UjFR808bB6ouEwfbi3k6bP/IQIrqqp
uW7ixIn+48ePJyEmsMABXsVtlOpq7248ZynsaponFY50YgQCw9nVd7dsNtUWq//Q
omXLGpFR6K27/m7VTB5d+SFGOH6/GUv4hZ0YSRWOpkxTmtRmPJvhAfanEqTW6v4i
WcZdlJtXfBVesi/k9u7+P4G1xLx+QhMIcSGHMrP1k3OlukIB73fsxCmv81ZV5Q1H
IIiI/bSHtElZApK3RSn5F1FS9nmp5quTSAkXtjsFCz5DIzz+ACXTsbyt7z1+XEiI
jPQGA2LWLlRWTW3dTH24q88/Dh/APCW/8wXtS8oylebRKPrSERsZMg5yqM5KUeTX
h0mty5Jlxo9Tj7vICDTGE8os5hqddfa0cxkGXUK8Xt9ZYkIIa8Lzwv5otCBEbl7v
z+fit4TZEonJJvn/sYGCL4fMZKn5gzTww4+o6ts8MqsgCVE8hacEgwMDrLJ0vAB6
bW1dCwOmQGW5FI97olBZHC9GxiaUQz2Yf+Rz2rTsSnLglRpbEu/WINV9erotrBIu
FLcw0XlvNcnPk6xmwhvwT+GEWPXDQofCE0JdXWVZy1sOp84frryXlZzIS4cpPdd5
gKXDmx+vDD/+NjFn7PntnQZwIQhhufDJIS8SFXf5GQm8ECL3u/qkTWtWhXEPbzxM
8RURUidUjt2wYldAmYgkGX4JJIu5RvwuWy1H6ISwVHggGUI6sNQ7She6fQr5n+By
jSK0qRmK39NLkU1dNHhXL2nu0Zc9yib9gbPl77yiUVgtFphjqsG4uBTMtE0JUThU
FyHLlH7Ed8PlBn9cT7WfOZuC320g1wlDh/K7i4lbIxR75Kgx8xlxyeMyg9BGEABo
JU0V6kUgxpMTfkMSEcwazqfolIib/C+GqG7JNKrmuQf16z0lsyBBqTmObQG8AmUD
bwNzYT50Qvixrik+Y0CIyScT7307QNXfbDaD0vPyvmxhxleMRwiGxB3qS7fduk7J
QDKgsrIRIwUm/mdh6nvrQ8pMHwq4Oik7JHYfqJyZ+nAYCrylqTAEQSy26BG6qjLE
Z7iybPFqlUrxv+KJIc+Doi92izj3Nj+5etyUmZskbYZBSMJF3u0BW95KvrBirRqE
CAlh2RAiIxa6YGBG6JQBhaq/wbPww9nFtExbgtLX8WzUIMP1ET+8WTGFXP26UI6w
inGZXC4sCnVllxBdXeGurayNBAXMxr2fO4vS8+KUvC5GqYWDpDWo5H7LT/72EPl+
w2tgpz5eGEmTPuZpYSaygMj5n96lMR42JMOSZlQ35NkRIByxST3iAVXg6RB538i/
IDmqCsZRJhshhlEfExsyjjAq661IIYDmkOTohLCE8D+Y6I05VREp6y2d2ZWJGbpx
CxJ/QYiK10tZZDTM37Jq7ZTutur+JrEc7zpZfJjryvCrRP9zVkl1ucQT55KSnhGJ
+H8UyQXpQGsNk5GVEASLAdbYMKL0FifCRIyrUwZEM8ygivDwfUNNSbSlT0cCRt1R
WWXtByDCxBx+npzDmRICA2OKj4hyfk47AhhZGQvCNqMudBXEB2JkCtFpb06FVwCk
BeAGDhYJwbRdio9DSPn6iY45iBD/Ry0q1lWWkA65ZKIzVb5mVW5N5jyEJcU27MWQ
S+gwwYVDSLm6iJAQ+VAR9oKdLiFYfjem7o4JKRcdGPZi/sf1MSk2oy7Gw3LY6zBS
HkZYIKTdFrZd018EERIijLohMmKoVZ4mVXwtUioAhPQb8xDYEMOoOyakbB3FxByE
GBpKEAKpkAHOsLdcfGQxFzXajDoIkSrLsSHlYsSCOdv2oYToKitLTNnaVZEVwTLY
VJZhMnSjjtca5bqKY0PK1kHMqQaGvdbFRWFDIDL4M1VX2dpVsRWZWAvcdQ2lj7KY
BFN8HAkpTwexkCAqtKosiI4cZQkxKU+TnFrwHIpRwFvyUlrMYS8zwpFYWHFEpDw9
JTuyFWRY5yH6TB3NyCYqT6MquxYpFUDBNuwV0iF0GL4960hIubqJabe5Qom7btSh
y8RqLxPiqKxy8WFijgolOZZRFiTDUVllY0OQoGOOOu0SIoa9uqqSEeVsWKXWZcPa
btSNmTqYclRWWfoHTLU5Uwfu1nkIm3hTZBxCysKHqMQqIXiLEc5iQ+Tiogh3fk47
AhACHXNUJcnRCeEHuyIAYuSorNNOhaxArI4Yc3E7IZaXHHTTLrM459OJgBQC1CGX
roSEsHiYIuPMC08nBfayrSoLHMAJQvDvCFJk5Nme1bk6HQhYsR5q1MU42FFYpwP4
QmVmh73ZJSuLyjIsvqOzCuE3tuEYQJlY48I67DWWe4V8mInGtn6ntKEIgBD5wION
umBEH/ay7LAzI4dmdULGHgFdKmT/5+33xAf5QQjoyEjxEVIy9rU7JeYiwI8J8ca7
lBDmAB/51SQhSRAiJoXDMoIEKMZxwyEgO3m+dPhUoo65jmZGTeNDlzohLCKDsPj6
s9182S1hDh8WMIbxWpZGclO6+ZPkwFz27UwqdZLTaPjwiMbb353AxAQiVIxVUahU
erk1ONdDEGC7MCRMBvA3FnUJYcyBezye6OE4nZBEMt4tjDzUFrPq5w8AF3JpZ3e2
QtAMCedNJYeEyQD+NK/AGnhDCAYGYgc4ThCi9scG9yMQpODcOBFfvc7vUvy9WceV
hkAmNfTbYTJnIBAQWANvqK5jR4/v4zhVqKyeQ4d3YXFLHpN5D4xCLsnbqDquNATS
vAFnIVddU23iDVJSgzFsSaFLyCtvvLaPRYdHWpAQlWa0tBQqh7OUYGcK566YGEiH
mjb3+hpy3828w4TEG+dwOGwadaLBwXgmnT4pxIcBnzR58pACrAGxo7A/jiuGQJz3
CCnmpp9zjlVlxffs2YPNi3Wjzp447+Z8CLoMastf4PvusgKwz8M0eemccxCAqkoV
Ue3Ad2JTk7AdEIL+WOxX0Wg0jmLk9/aS3V2H/12XEP4KHI+Rzz3vkznV2C9jR7tF
gfZQ50rjf+2IHSuuQZonQV3pqh9CcORIVzsjJ0YAkpD0umef3cTTd3whXtiJyy6/
vCi6KDDS3cV6svDQrmgB4zASWESOdBW9M8zQP7NwoSAEWDMhqd0dHf/LmQSQkhDM
YMK8Ef0bWPjFQmNDY+OwtgTsRnqOUDLGs34QWcEu2R8THRQdtZibUF9PvN8Uw6VP
MSKR8HO7d++GSIlZpCQEZQyse+aZbwg7Aub4uHrxkmG3gAMRA70nRWMSvElJJdkW
3GsiEqFw1yEaOAmbPLy74aZl+oqImKGr6Y7t2x/nXFhYFE5/QKX7MydOnOiORiMb
QjWhGxEEJi9ua6NtW4ffJVrlPWIHw7z9HR8QS8XtIRfvn6Hvg8uLkWf6eiQ6vuio
3LN5SUTNYLuJ4tKgw5r9veiii6k2xBvdGPmikfC6zZs37+IUpt7PhcnDI4BzVt5x
x25OJMiC+lrf3k5Huorrxmy1ji8fAk3NzfSnX/qSGcW4nnzphReuOHjw4PscaA5Z
c7/Dxx09k6wKBnfwxGU5XtHiHV/pPB5x7du3l6crpmSZBTue4RFoaGigm1esEFiK
194Y132dnXdu27btd5zbNp3PJQSlZw4cOHB0xowWb3V19QIEKIqLzp87V0gJj5cR
5LgSEcCqx41sN4ChdH3hvp+2P//8o3yNHTVtei8fIUiQ7OjYub11dutMn983GwXB
LrS2tooJTWdnJ4IcVwQB4HXN0qW0YMECxk4m1GgwPvjmk2vWfJ1DjvMxZOuhfIQg
N4Zg8ffee/fN2bNnT/P5/K3MI/6bh+p5u+q2+fN5/T5Ovb19PGIYUibyV6zz8lZ5
c+bMoeXLl1NDPW8rhe5tHAP9/a+uWb36Fg7hnW2yhtwKlsmdNdDix64vzcuWLbur
qanpr5l1fZMpkQBZNdq/fz/t3LmT+vr6hI3hJRhL9vHvxYOmYDDI297V0gUXXEAt
rKLsTuCU7unp+cGGDRse4zjsWmwacXva0gajGG3VX3jhhfPb2toe4h4gVFhuQfIa
hj/CY/MBfm4CKUrxWB1PzuSbeTLdmXbmTbt4SclDHq+H8Cyjinc8BQkgo5hLJpN7
tmzZcndHR8c7nA5L7EV77HASIuvCBBJ7rjZfe+21K6ZMmXKHx+3RH5rogiLT2SmG
qBZzw+XNjbeWlRtX6jXSwVnbVkpePZc9H8IK5M1k0l3dPT2Pbdy4cR2ngorCgyQx
G+dzQSebVzCBJQJpIS3YNbJp0aJFi6dOnnpzoCpwGUeMpBxLkePLyxxr8YGB33V3
H33m9TdffzWRSEA9YTNcSIW1CxS88dEAKYmBrNZNZtfS0tLKs/p5daHQfH8gMIdX
i/nD7+Pf8aTtGKvlXeFodBvvhb6d7ekuthVYl8LDEEzaSiZCojUaQmRenKHKIDUw
9tjqUxw+ny/Q2NhYzfYmwEbPz9ceHhC4ec92Oao71Xq5qrI40at5Vs1rrlqa7QGP
WdIJtotxXmbq52s8w8DETh5YQgcJw6omTpPXjTUwKE8eIEv65RmNgP9McyAm9wDo
1rAxuaf/Bz7+WCRMSO02AAAAAElFTkSuQmCC" /> </td>
        <td align="left" valign="middle"><h4 style="margin-top:5px;">🐧🍏🪟 Terminal Command</h4></td>
    </tr>
</table>
<blockquote>
docker run --privileged --rm tonistiigi/binfmt --install arm64<br />
docker run --rm --privileged multiarch/qemu-user-static --reset -p yes<br />
docker run -it --platform linux/arm64 python:3-slim uname -a
</blockquote>

That final command will run the `python` docker image with `--platform linux/arm64` mode and output the result of running `uname -a` which on my machine outputs:

```bash
Linux da5febed1186 5.15.49-linuxkit #1 SMP Tue Sep 13 07:51:46 UTC 2022 aarch64 GNU/Linux
```

Here we can see `aarch64` which is the linux way to describe `arm64`.

So, we can just do the reverse for `Apple Silicon` right?

😭 Unfortunately not.

### CPU Optimizations

Unfortunately, while `docker` / `qemu` can emulate basic ops which most CPUs use, it can't handle things like `AVX` (Advanced Vector Extensions) a special type of instruction set designed to run `tensor` operations faster on `x86` CPUs.

What you end up finding is that many of the specialized `deep learning` / `data science` libraries we use are compiled for each platform to be as fast as possible.

Unfortunately that means we often can't use emulated `x86` mode, so we need native compiled binary versions for `arm64` and the operating system we want them to run on.

So, we can just use the `Apple Silicon` packages right?

😭 Yes and No. Because `syft` runs in multiple contexts we actually need a set of packages for the `client` operating system which could be `macOS` running on Apple Silicon and for the `server` operating system which is always `linux` because our containers are linux containers.

### JAX

Let's use `jax` as an example of this issue to see how we solved it. JAX is an awesome DL library from Google. To make it super fast and support all the best hardware `jax` is split into at least two parts, one which provides all the non compiled and optimized code called `jax` and another which has the OS and CPU / GPU specific code called `jaxlib`.

If you check `PyPI` you will see a list of the different `python` / `OS` / `CPU` combinations that `jaxlib` has been compiled for.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://pypi.org/project/jaxlib/0.3.14/#files">jaxlib 0.3.14 Downloads</a></blockquote>

What you will see is that while there are `arm64` wheels for `macOS` there are no `aarch64` wheels for Arm64 on linux.

To solve this we had to compile our own `arm64` linux version of `jax` without any of these special `x86` only CPU extensions.

For now this has been documented as a series of steps which can be run manually on an `x86` cloud instance to compile it under emulation. Last time this took about 4 hours on a 32-core machine. It's probably faster on actual Apple Silicon but I haven't tried.

<blockquote><strong>🔗 Web Link</strong><br /><a href="https://github.com/OpenMined/PySyft/blob/dev/packages/grid/backend/JAX.MD">JAX.md</a></blockquote>

Once we have that pre-built wheel this is how we use it.

```dockerfile
# packages/grid/backend/backend.dockerfile

# copy precompiled arm64 packages
COPY grid/backend/wheels /wheels

# apple m1 build PyNaCl for aarch64
RUN --mount=type=cache,target=/root/.cache if [ $(uname -m) != "x86_64" ]; then \
  # precompiled jaxlib and dm-tree
  pip install --user /wheels/jaxlib-0.3.14-cp310-none-manylinux2014_aarch64.whl; \
  ...
```

Here you can see by using an `if [ $(uname -m) != "x86_64" ];` bash statement we can execute code that will only run during `arm64` builds.


Later when `syft` installs inside the container and asks for `jax==0.3.14` it will find that its already installed and continue along happily.

We do more similar things with other packages either to speed up build times (dm-tree) or to provide our own manually compiled `aarch64` linux compatible compiled wheels to support things like `tensorflow-federated` on linux arm64, which is still a work in progress.

It would be great to eventually upstream automated `aarch64` builds to the various projects we rely on, or at the very least have our own `CD` pipeline for them.

```dockerfile
# packages/grid/backend/backend.dockerfile

  ...
  tar -xvf /wheels/dm-tree-0.1.7.tar.gz; \
  pip install --user pytest-xdist[psutil]; \
  pip install --user torch==1.11.0 -f https://download.pytorch.org/whl/torch_stable.html; \
  git clone https://github.com/pybind/pybind11 && cd pybind11 && git checkout v2.6.2; \
  pip install --user dm-tree==0.1.7; \
  # fixes apple silicon in dev mode due to dependency from safety
  pip install --user ruamel.yaml==0.17.21; \
  pip install --user /wheels/tensorstore-0.1.25-cp310-cp310-linux_aarch64.whl; \
  # pip install --user tensorflow-aarch64==2.10.0; \
  # pip install --user /wheels/tensorflow_compression-2.10.0-cp310-cp310-linux_aarch64.whl; \
  fi
```

## Cake

## Storage / Persistence

## Inventory

In [ ]:
Homework
test out quickstart
try a bunch of tutorials?